In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import tensorflow as tf
import pathlib
import matplotlib.pyplot as plt
import datetime

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv3D, Flatten, Dropout, MaxPooling3D, GlobalMaxPooling3D

from generators import VideoDataGenerator
from loader import get_classes

AUTOTUNE = tf.data.experimental.AUTOTUNE
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

print("Tensorflow Version: ", tf.__version__)
print("Num GPUs Available: ", len(gpus))

In [ ]:
data_dir = pathlib.Path("./data")
classes = get_classes(data_dir / "classes.txt")

params = {
    'color_mode': 'landmarks',
    'optical_flow': False,
    'batch_size': 8,
    'shuffle': True,
    'classes': classes
}

partition = {
    'train': list((data_dir / "Train").glob("**/*.mp4")),
    'test': list((data_dir / "Test").glob("**/*.mp4"))
}

training_generator = VideoDataGenerator(partition['train'], **params)
validation_generator = VideoDataGenerator(partition['test'], **params)

In [ ]:
model = Sequential([
    Conv3D(16, (5, 3, 3), activation='relu', input_shape=(training_generator.dim)),
    MaxPooling3D(),
    Conv3D(32, (3, 3, 3), activation='relu'),
    MaxPooling3D(),
    Conv3D(64, (1, 3, 3), activation='relu'),
    MaxPooling3D((1,2,2)),
    Flatten(),
    Dense(2048, activation='relu'),
    Dense(1024, activation='relu'),
    Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(training_generator, validation_data=validation_generator, epochs=100, callbacks=[callback])

In [ ]:
#plt.plot(history.history['accuracy'], label='accuracy')
#plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Accuracy/Loss')
plt.ylim([0, 1])
plt.legend(loc='lower right')

In [ ]:
export_time = datetime.datetime.now()
export_path = pathlib.Path("export/model_{}.h5".format(export_time.strftime("%Y-%m-%d-%H%M")))
export_fig_path = pathlib.Path("export/model_{}_hist.svg".format(export_time.strftime("%Y-%m-%d-%H%M")))
print("Exporting trained model to {}".format(export_path.absolute()))

plt.savefig(export_fig_path, format='svg', bbox_inches='tight')
model.save(export_path)